In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
def csv_ensemble(csv_paths, save_dir):  
    def decode_rle_to_mask(rle, height, width):
        # RLE로 인코딩된 결과를 mask map으로 복원합니다.
        s = rle.split()
        starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
        starts -= 1
        ends = starts + lengths
        img = np.zeros(height * width, dtype=np.uint8)
        
        for lo, hi in zip(starts, ends):
            img[lo:hi] = 1
        
        return img.reshape(height, width)

    def encode_mask_to_rle(mask):
        # mask map으로 나오는 인퍼런스 결과를 RLE로 인코딩 합니다.

        pixels = mask.flatten()
        pixels = np.concatenate([[0], pixels, [0]])
        runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
        runs[1::2] -= runs[::2]
        return ' '.join(str(x) for x in runs)

    csv_column = 8352

    csv_data = []
    for path in csv_paths:
        data = pd.read_csv(path)
        csv_data.append(data)

    file_num = len(csv_data)
    filename_and_class = []
    rles = []

    for index in tqdm(range(csv_column)):    
        model_rles = []
        for data in csv_data:
            if(type(data.iloc[index]['rle']) == float):
                model_rles.append(np.zeros((2048,2048)))
                continue
            model_rles.append(decode_rle_to_mask(data.iloc[index]['rle'],2048,2048))
        
        image = np.zeros((2048,2048))

        for model in model_rles:
            image += model
        
        image[image <= (file_num // 2)] = 0
        image[image > (file_num // 2)] = 1

        result_image = image

        # 앙상블 결과 rle로 인코딩
        rles.append(encode_mask_to_rle(result_image))
        filename_and_class.append(f"{csv_data[0].iloc[index]['class']}_{csv_data[0].iloc[index]['image_name']}")

    # to csv
    classes, filename = zip(*[x.split("_") for x in filename_and_class])
    image_name = [os.path.basename(f) for f in filename]

    df = pd.DataFrame({
        "image_name": image_name,
        "class": classes,
        "rle": rles,
    })

    df.to_csv(save_dir, index=False)

In [ ]:
csv_path1 = 'CSV 파일 경로 1'
csv_path2 = 'CSV 파일 경로 2'
csv_path3 = 'CSV 파일 경로 3'

csv_paths = [csv_path1, csv_path2, csv_path3]
save_dir = './ensemble.csv'

In [ ]:
csv_ensemble(csv_paths, save_dir)